In [2]:
import xarray as xr
import numpy as np
import pyproj
import s3fs

fname = 'hrrr_validate/hrrr.t11z.wrfsfcf02-TMP:202405061300:0 in surface:forecast-63.grib2'


ds_cfgrib = xr.open_dataset(fname, engine='cfgrib')
ds_cfgrib

<xarray.Dataset>
Dimensions:     (y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    valid_time  datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    t           (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-05-06T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
ds_gribberish = xr.open_dataset(fname, engine='gribberish')
ds_gribberish

/Users/matthew/Developer/gribberish/python/gribberish/gribberish_backend.py:51: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  return xr.Dataset(


<xarray.Dataset>
Dimensions:    (time: 1, y: 1059, x: 1799)
Coordinates:
  * time       (time) datetime64[ns] 2024-05-06T13:00:00
  * y          (y) float64 -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
  * x          (x) float64 -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
Data variables:
    tmp        (time, y, x) float64 ...
Attributes:
    meta:     Generated with gribberishpy

Verify that the coordinates match between cfgrib and gribberish

In [4]:
lat_cfgrib = ds_cfgrib.latitude.values
lon_cfgrib = ds_cfgrib.longitude.values
lat_gribberish = ds_gribberish.latitude.values
lon_gribberish = ds_gribberish.longitude.values

np.allclose(lat_cfgrib, lat_gribberish), np.allclose(lon_cfgrib, lon_gribberish + 360.0)

(True, True)

In [5]:
to_xy = pyproj.Transformer.from_crs('epsg:4326', ds_gribberish.tmp.crs, always_xy=True).transform

In [6]:
target_lat = 41.434473
target_lon = -71.453197

x, y = to_xy(target_lon, target_lat)
x, y

(2145435.489970598, 632072.4556247981)

In [42]:
ii_x = (x - -2697520.14252193) / 3000.0
ii_y = (y - -1587306.152556663368766531) / 3000.0
ii_x, ii_y

(1614.3185441641758, 739.7928693938205)

In [41]:
ds_gribberish.y.values

array([-1587306.15255666, -1584306.15255666, -1581306.15255666, ...,
        1580693.84744334,  1583693.84744334,  1586693.84744334])

In [38]:
i_x = list(ds_gribberish.x.values).index(ds_gribberish.sel(x=x, method='nearest').x)
i_y = list(ds_gribberish.y.values).index(ds_gribberish.sel(y=y, method='nearest').y)
i_x, i_y

(1614, 740)

In [34]:
i_x, i_y = 1615, 737

In [35]:
temp = ds_gribberish.tmp.isel(x=i_x, y=i_y).values 
# convert from Kelvin to Fahrenheit
#temp = (temp - 273.15) * 9/5 + 32
temp

array([283.13996887])

In [36]:
temp = ds_cfgrib.t.isel(x=i_x, y=i_y).values 
# convert from Kelvin to Fahrenheit
#temp = (temp - 273.15) * 9/5 + 32
temp

array(283.13995, dtype=float32)

In [ ]:
a: 283.1455078125
b: 283.2080078125
c: 283.2080078125
d: 283.2705078125
value: 283.20987057910645

